In [1]:
%run 'GPDM_direct_fixedpoints.ipynb'

/nfs/nhome/live/gbohner/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



In [ ]:
3

In [2]:
import scipy.optimize
import pickle
import pickle, datetime, time

In [56]:
with open('well_1d_k2_20171129T104605.pkl', 'r') as f:
    [kwells_params, x, y, all_results, init_paramvec, dict_ind, dict_shape, opt_params] = pickle.load(f)

In [57]:
plots_by_run = []
for v in range(Ny):
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(T)), 
                      y=np.squeeze(y[:,:,v]), 
                      mode='lines')
    )
    
plt(plots_by_run)

In [84]:
def kwells_true_tr_fnc(xprev_vec, well_locs, Sigma_eps, well_width):
    xprev_vec = np.atleast_2d(xprev_vec)
    out = np.zeros_like(xprev_vec)
    for i in range(np.prod(xprev_vec.shape)):
        xprev=np.asscalar(xprev_vec[:,i])
        
        in_well = False
        for well_loc in well_locs:
            if ((xprev>(well_loc - np.sqrt(Sigma_eps)*well_width)) and (xprev<(well_loc + np.sqrt(Sigma_eps)*well_width))):
                out[:,i] = well_loc
                in_well = True
                
        if in_well == False:
            out[:,i] = xprev

    return out

In [85]:
# # Plot smoothed trajectories
obj, x_t1, x_t, sig_t1, sig_t = time_full_iter(all_results[5].x,
                                                y, dict_ind, dict_shape, ret_smoothed=True)



In [86]:

plots_by_run = []
for v in [1]:
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(T)), 
                      y=np.squeeze(x[:,:,v]), 
                      mode='lines', name='x_true')
    )
    
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(T)), 
                      y=np.squeeze(x_t1[:,:,v]), 
                      mode='lines', name='x_hat_tminus1')
    )
        
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(T)), 
                      y=np.squeeze(x_t[:,:,v]), 
                      mode='lines', name='x_hat_t')
    )
    
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(np.arange(T)), 
                      y=np.squeeze(y[:,:,v]), 
                      mode='lines', name='y_true')
    )
    
plt(plots_by_run)

In [87]:
np.sqrt(all_results[5].x[dict_ind['Sigma_nu']])

array([ 0.09531614])

In [88]:
# Examine parameters
it = -1

np.vstack([all_results[it].x[dict_ind['z']], all_results[it].x[dict_ind['u']], np.sqrt(all_results[it].x[dict_ind['Sigma_u']])]).T


array([[-1.61394161, -1.57900674,  0.46739495],
       [-1.43714543, -1.14941222,  0.31125681],
       [-1.04999464, -1.11472806,  0.22359349],
       [-0.90559326, -1.03520325,  0.22182877],
       [-0.61933323, -0.73974494,  0.26009201],
       [-0.33480261, -0.30669052,  0.39131407],
       [-0.25053043, -0.43060682,  0.31957226],
       [-0.03155219, -0.0139922 ,  0.21642708],
       [ 0.2724219 ,  0.37877371,  0.08640889],
       [ 0.49354751,  0.37158259,  0.07849529],
       [ 0.70756097,  0.64239609,  0.20669795],
       [ 0.94697736,  0.92196123,  0.25476929],
       [ 1.22566627,  1.28839742,  0.33761913],
       [ 1.35633688,  1.35569443,  0.18109467],
       [ 1.52205879,  1.63385393,  0.22654422],
       [ 1.83835023,  1.8304613 ,  0.35888573],
       [ 2.11625071,  2.24431713,  0.37811544],
       [ 2.40779415,  2.42144168,  0.36750053],
       [ 2.61329446,  2.48970907,  0.25319734],
       [ 2.83216456,  2.78054397,  0.36009418]])

In [89]:
def replace_params(pvec, opt_params, paramvec):
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds,opt_params)
    out = np.concatenate([pvec, paramvec[inds]])
    out = out[np.argsort(np.concatenate([opt_params, inds]))]
    return out

In [90]:
# Define a plotting function for callback that shows the current transition function estimate
def kwells_callback_plot(pvec_partial):
    
    pvec = replace_params(pvec_partial, opt_params, init_paramvec)
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J) = \
        vec_to_params(pvec, dict_ind, dict_shape)

    # Plot transition function
    xstar = np.atleast_2d(np.arange(np.min(z)-0.5,np.max(z)+0.5,0.05))

    fp_get_static_K, fp_predict = create_fp_gp_funcs()
    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u=Sigma_u)
    mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

    # print(time_full_iter(pvec, y, dict_ind, dict_shape)[0])
        
    # Get true function values
    true_tr_vals = kwells_true_tr_fnc(xstar, kwells_params["well_locs"], 
                                                          kwells_params["Sigma_eps"], 
                                                          kwells_params["well_width"])
    
    
    plt([plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star), mode='markers', name='GP post. mean',
                         marker=dict(color='blue')),
         plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star+np.sqrt(np.diag(sig_star))), mode='markers', 
                         marker=dict(size=2, color='blue')),
         plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star-np.sqrt(np.diag(sig_star))), mode='markers', 
                         marker=dict(size=2, color='blue')),      
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange')),
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals+np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange', size=2)),
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals-np.sqrt(kwells_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange', size=2)),
         plt_type.Scatter(x=np.squeeze(z), y=np.squeeze(-2.0*np.ones_like(z)), mode='markers', marker=dict(size=10),
                         name = 'Ind point loc'),
         plt_type.Scatter(x=np.squeeze(z), y=np.squeeze(u), mode='markers', marker=dict(size=10),
                         name = 'Ind point val'),
         plt_type.Scatter(x=np.atleast_1d(np.squeeze(s)), y=np.atleast_1d(np.squeeze(s)), mode='markers', marker=dict(size=10),
                         name = 'Fixed point')
        ])

In [93]:
3

3

In [95]:
kwells_callback_plot(all_results[-1].x)

In [71]:
xstar = np.atleast_2d(np.arange(np.min(z)-0.5,np.max(z)+0.5,0.05))


In [49]:
pvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J) = \
    vec_to_params(pvec, dict_ind, dict_shape)

# Plot transition function
xstar = np.arange(np.min(z)-0.5,np.max(z)+0.5,0.05)

fp_get_static_K, fp_predict = create_fp_gp_funcs()
L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                     sig_eps=Sigma_eps, sig_u=Sigma_u)
mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

In [92]:
[a.fun for a in all_results]

[array([[ 47.36484115]]),
 array([[ 45.56983218]]),
 array([[ 44.18495326]]),
 array([[ 43.8244072]]),
 array([[ 42.84083256]]),
 array([[ 42.31965319]]),
 array([[ 41.87879662]]),
 array([[ 41.53107878]]),
 array([[ 41.23908672]]),
 array([[ 41.01554429]]),
 array([[ 40.57629551]]),
 array([[ 40.34345735]]),
 array([[ 40.19382725]]),
 array([[ 40.12188542]]),
 array([[ 40.00004537]]),
 array([[ 39.94638807]]),
 array([[ 39.09780572]]),
 array([[ 38.79849607]]),
 array([[ 38.7588423]]),
 array([[ 35.76868956]]),
 array([[ 34.73623412]]),
 array([[ 34.68048682]]),
 array([[ 34.64193423]]),
 array([[ 34.58043939]]),
 array([[ 34.54731442]]),
 array([[ 34.43994299]]),
 array([[ 34.38537559]]),
 array([[ 34.35591061]]),
 array([[ 30.23666607]]),
 array([[ 29.36415172]]),
 array([[ 29.15838459]]),
 array([[ 29.03523886]]),
 array([[ 28.51557928]]),
 array([[ 28.45351674]]),
 array([[ 28.34846585]]),
 array([[ 28.30802779]]),
 array([[ 28.2584767]]),
 array([[ 28.24438386]]),
 array([[ 27.97